In [9]:
import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np

# Load the uploaded CSV files
log_episode_path = 'D:\\Aprendizado_por_Reforco\\TradeM1\\4.18.4\\log_episode_25.csv'
bar_m15_path = 'D:\\dados\\bar_M15_V02_data_01-01-2023_a_31-08-2024.csv'
df = pd.read_csv(log_episode_path)

log_episode_df = pd.read_csv(log_episode_path)
bar_m15_df = pd.read_csv(bar_m15_path)

# Convert DateTime to pandas datetime format in bar_m15_df
bar_m15_df['DateTime'] = pd.to_datetime(bar_m15_df['DateTime'])

# Map entry and exit DateTime to log_episode_df based on entry_step and exit_step
log_episode_df['entry_datetime'] = log_episode_df['entry_step'].apply(lambda x: bar_m15_df.iloc[x]['DateTime'] if x < len(bar_m15_df) else pd.NaT)
log_episode_df['exit_datetime'] = log_episode_df['exit_step'].apply(lambda x: bar_m15_df.iloc[x]['DateTime'] if x < len(bar_m15_df) else pd.NaT)

# Use entry_datetime as the DateTime for plotting
log_episode_df['DateTime'] = log_episode_df['entry_datetime']

# Plot the profit progression over time
fig_profit = go.Figure()
fig_profit.add_trace(go.Scatter(
    x=log_episode_df['DateTime'],
    y=log_episode_df['profit'],
    mode='lines',
    name='Profit Progression',
    line=dict(color='blue')
))

# Set chart title and labels for profit progression
fig_profit.update_layout(
    title='Profit Progression Over Time',
    xaxis_title='Date',
    yaxis_title='Profit',
    template='plotly_white'
)

# Save the profit progression plot to a folder
output_folder = '4.5'
os.makedirs(output_folder, exist_ok=True)
fig_profit.write_html(os.path.join(output_folder, 'profit_progression.html'))

# Show the profit progression plot
fig_profit.show()

# Plot the cumulative profit over time
log_episode_df['cumulative_profit'] = log_episode_df['profit'].cumsum()

fig_cumulative_profit = go.Figure()
fig_cumulative_profit.add_trace(go.Scatter(
    x=log_episode_df['DateTime'],
    y=log_episode_df['cumulative_profit'],
    mode='lines',
    name='Cumulative Profit',
    line=dict(color='green')
))

# Set chart title and labels for cumulative profit
fig_cumulative_profit.update_layout(
    title='Cumulative Profit Over Time',
    xaxis_title='Date',
    yaxis_title='Cumulative Profit',
    template='plotly_white'
)

# Save the cumulative profit plot to a folder
fig_cumulative_profit.write_html(os.path.join(output_folder, 'cumulative_profit.html'))

# Show the cumulative profit plot
fig_cumulative_profit.show()

# Calculate metrics for individual trades
def calculate_drawdown(profit_series):
    cumulative = profit_series.cumsum()
    high_watermark = cumulative.cummax()
    drawdown = cumulative - high_watermark
    return drawdown

drawdown = calculate_drawdown(log_episode_df['profit'])
max_drawdown = drawdown.min()

# Sharpe Ratio per operation
average_profit = log_episode_df['profit'].mean()
profit_std = log_episode_df['profit'].std()
sharpe_ratio = average_profit / profit_std if profit_std != 0 else 0

# Sortino Ratio per operation
negative_profit = log_episode_df[log_episode_df['profit'] < 0]['profit']
negative_std = negative_profit.std()
sortino_ratio = average_profit / negative_std if negative_std != 0 else 0

# Winrate calculation
num_wins = len(log_episode_df[log_episode_df['profit'] > 0])
num_losses = len(log_episode_df[log_episode_df['profit'] <= 0])
num_trades = len(log_episode_df)
winrate = (num_wins / num_trades) * 100 if num_trades > 0 else 0
# Total profit
total_profit = log_episode_df['profit'].sum()

# Additional trade metrics
profit_factor = log_episode_df[log_episode_df['profit'] > 0]['profit'].sum() / abs(log_episode_df[log_episode_df['profit'] <= 0]['profit'].sum())
expected_payoff = total_profit / num_trades if num_trades > 0 else 0
average_win = log_episode_df[log_episode_df['profit'] > 0]['profit'].mean()
average_loss = log_episode_df[log_episode_df['profit'] <= 0]['profit'].mean()
recovery_factor = abs(total_profit / max_drawdown) if max_drawdown != 0 else 0

# Period-based metrics
# Calculate the cumulative profit over time (daily or desired time interval)
log_episode_df['daily_return'] = log_episode_df['cumulative_profit'].pct_change()

# Remove NaN values (which arise from the first pct_change calculation)
daily_returns = log_episode_df['daily_return'].dropna()

# Sharpe Ratio for the entire period
average_daily_return = daily_returns.mean()
daily_return_std = daily_returns.std()
sharpe_ratio_period = average_daily_return / daily_return_std if daily_return_std != 0 else 0

# Sortino Ratio for the entire period
negative_daily_returns = daily_returns[daily_returns < 0]
downside_std = negative_daily_returns.std()
sortino_ratio_period = average_daily_return / downside_std if downside_std != 0 else 0

# Print the results
print(f"Max Drawdown: {max_drawdown}")
print(f"Sharpe Ratio (per operation): {sharpe_ratio}")
print(f"Sortino Ratio (per operation): {sortino_ratio}")
print(f"Sharpe Ratio (entire period): {sharpe_ratio_period}")
print(f"Sortino Ratio (entire period): {sortino_ratio_period}")
print(f"Winrate: {winrate:.2f}%")
print(f"Profit Factor: {profit_factor}")
print(f"Expected Payoff: {expected_payoff}")
print(f"Average Win: {average_win}")
print(f"Average Loss: {average_loss}")
print(f"Recovery Factor: {recovery_factor}")
print(f"Total Trades: {num_trades}")
print(f"Winning Trades: {num_wins}")
print(f"Losing Trades: {num_losses}")
print(f"Total Profit: {total_profit}")


Max Drawdown: -1319.0
Sharpe Ratio (per operation): 0.07242329212780521
Sortino Ratio (per operation): 0.08824765167843138
Sharpe Ratio (entire period): 0.01939350710851944
Sortino Ratio (entire period): 0.026297084017348704
Winrate: 53.67%
Profit Factor: 1.3028722408548474
Expected Payoff: 4.415150653780557
Average Win: 35.39009533898305
Average Loss: -31.462576687116563
Recovery Factor: 5.887983320697498
Total Trades: 1759
Winning Trades: 944
Losing Trades: 815
Total Profit: 7766.25
